# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score

# from sklearn.metrics import confusion_matrix
# 
# from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /Users/stingl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stingl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stingl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
connection = engine.connect()
df = pd.read_sql_table("messages", con=connection)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] 

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators = 10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
def show_metrics(Y_test, Y_pred):
    """
    Print f1-score, precision and recall for each category
    
    Parameters:
    -----------
    Y_test: True values,
    Y_pred: Predicted values
    """
    for cat in range(0, Y_test.shape[1]):
        cf=classification_report(Y_test.iloc[:, cat], pd.DataFrame(Y_pred).iloc[:, cat], output_dict=True).get('1')
        if cf is not None:
            print(f"Category:{cat: 3d}, f1-score: {cf['f1-score'] :5.3f}, precision: {cf['precision']: 5.3f}, recall: {cf['recall']: 5.3f}")
        else:
            print(f"Caterogy:{cat: 3d} doesn't have value 1")
                  
Y_pred = pipeline.predict(X_test)                  
show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.879, precision:  0.842, recall:  0.920
Category:  1, f1-score: 0.564, precision:  0.761, recall:  0.448
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.678, precision:  0.756, recall:  0.614
Category:  4, f1-score: 0.157, precision:  0.604, recall:  0.090
Category:  5, f1-score: 0.187, precision:  0.786, recall:  0.106
Category:  6, f1-score: 0.055, precision:  0.500, recall:  0.029
Category:  7, f1-score: 0.012, precision:  0.143, recall:  0.006
Category:  8, f1-score: 0.103, precision:  0.600, recall:  0.056
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.337, precision:  0.813, recall:  0.213
Category: 11, f1-score: 0.563, precision:  0.852, recall:  0.420
Category: 12, f1-score: 0.412, precision:  0.811, recall:  0.276
Category: 13, f1-score: 0.176, precision:  0.923, recall:  0.098
Category: 14, f1-score: 0.098, precision:  0.769, recall:  0.052
Category: 15, f1-score: 0.104, precision:  0.625, recall

In [24]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88      5941
           1       0.76      0.45      0.56      1333
           2       0.00      0.00      0.00        34
           3       0.76      0.61      0.68      3286
           4       0.60      0.09      0.16       644
           5       0.79      0.11      0.19       414
           6       0.50      0.03      0.06       239
           7       0.14      0.01      0.01       156
           8       0.60      0.06      0.10       267
           9       0.00      0.00      0.00         0
          10       0.81      0.21      0.34       512
          11       0.85      0.42      0.56       878
          12       0.81      0.28      0.41       714
          13       0.92      0.10      0.18       123
          14       0.77      0.05      0.10       192
          15       0.62      0.06      0.10        88
          16       0.56      0.07      0.12       259
          17       0.78    

In [21]:
cat=1
classification_report(Y_test.iloc[:, cat], pd.DataFrame(Y_pred).iloc[:, cat], output_dict=True).get('1')

{'precision': 0.7614795918367347,
 'recall': 0.44786196549137286,
 'f1-score': 0.5640056683986774,
 'support': 1333}

### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a28c4f6a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
               oob_score=False, random_state=Non

In [69]:
parameters = {
#     'vect__binary': [True, False], 
    'mcfl__estimator__max_depth': [20, 50, 100],
    'mcfl__estimator__min_samples_split': [2, 5, 20],
    'mcfl__estimator__n_estimators': [5, 10, 20]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   31.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   34.2s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.9min
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:  3.1min
[Parallel(n_jobs=12)]: Done  67 out of  81 | elapsed:  4.9min remaining:  1.0min
[Parallel(n_jobs=12)]: Done  76 out of  81 | elapsed:  5.6min remaining:   22.1s
[Parallel(n_jobs=12)]: Done  81 out of  81 | elapsed:  5.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__max_depth': [20, 50, 100], 'mcfl__estimator__min_samples_split': [2, 5, 20], 'mcfl__estimator__n_estimators': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [71]:
cv.best_params_

{'mcfl__estimator__max_depth': 100,
 'mcfl__estimator__min_samples_split': 5,
 'mcfl__estimator__n_estimators': 10}

In [72]:
cv.best_score_

0.6010371477330746

In [73]:
cv.score(X_test, Y_test)

0.5991757212439116

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [74]:
Y_pred = cv.predict(X_test)

show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.878, precision:  0.795, recall:  0.979
Category:  1, f1-score: 0.543, precision:  0.818, recall:  0.406
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.692, precision:  0.772, recall:  0.628
Category:  4, f1-score: 0.121, precision:  0.623, recall:  0.067
Category:  5, f1-score: 0.124, precision:  0.778, recall:  0.068
Category:  6, f1-score: 0.049, precision:  0.857, recall:  0.025
Category:  7, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  8, f1-score: 0.167, precision:  0.781, recall:  0.094
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.373, precision:  0.890, recall:  0.236
Category: 11, f1-score: 0.562, precision:  0.848, recall:  0.420
Category: 12, f1-score: 0.334, precision:  0.832, recall:  0.209
Category: 13, f1-score: 0.196, precision:  0.700, recall:  0.114
Category: 14, f1-score: 0.040, precision:  0.571, recall:  0.021
Category: 15, f1-score: 0.000, precision:  0.000, recall

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [75]:
print(classification_report(Y_test, Y_pred))

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       0.80      0.98      0.88      5941
           1       0.82      0.41      0.54      1333
           2       0.00      0.00      0.00        34
           3       0.77      0.63      0.69      3286
           4       0.62      0.07      0.12       644
           5       0.78      0.07      0.12       414
           6       0.86      0.03      0.05       239
           7       0.00      0.00      0.00       156
           8       0.78      0.09      0.17       267
           9       0.00      0.00      0.00         0
          10       0.89      0.24      0.37       512
          11       0.85      0.42      0.56       878
          12       0.83      0.21      0.33       714
          13       0.70      0.11      0.20       123
          14       0.57      0.02      0.04       192
          15       0.00      0.00      0.00        88
          16       0.50      0.02      0.04       259
          17       0.70    

In [77]:
pd.DataFrame(Y_pred)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
5,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [80]:
Y.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
    'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
#     'mcfl__estimator__degree' : [3, 4, 5, 6],
#     'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of   9 | elapsed:  6.2min remaining: 21.9min
[Parallel(n_jobs=12)]: Done   3 out of   9 | elapsed:  6.3min remaining: 12.5min
[Parallel(n_jobs=12)]: Done   4 out of   9 | elapsed:  6.6min remaining:  8.3min
[Parallel(n_jobs=12)]: Done   5 out of   9 | elapsed:  6.7min remaining:  5.4min
[Parallel(n_jobs=12)]: Done   6 out of   9 | elapsed:  6.7min remaining:  3.4min
[Parallel(n_jobs=12)]: Done   7 out of   9 | elapsed:  7.9min remaining:  2.3min
[Parallel(n_jobs=12)]: Done   9 out of   9 | elapsed:  8.0min remaining:    0.0s
[Parallel(n_jobs=12)]: Done   9 out of   9 | elapsed:  8.0min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__kernel': ['linear', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [122]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

{'mcfl__estimator__kernel': 'linear'}
0.6772289922751838
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5941
           1       0.78      0.58      0.67      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.69      0.72      3286
           4       0.65      0.20      0.31       644
           5       0.75      0.26      0.39       414
           6       0.79      0.13      0.22       239
           7       0.00      0.00      0.00       156
           8       0.68      0.25      0.37       267
           9       0.75      0.66      0.70       512
          10       0.79      0.76      0.78       878
          11       0.80      0.55      0.65       714
          12       0.76      0.45      0.56       123
          13       0.81      0.13      0.22       192
          14       0.57      0.18      0.28        88
          15       0.62      0.19      0.29       259
          16       0.78 

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [123]:
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
#     'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
    'mcfl__estimator__degree' : [2, 3, 4, 5, 6],
    'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  15 | elapsed:  7.3min remaining: 47.6min
[Parallel(n_jobs=12)]: Done   4 out of  15 | elapsed:  7.4min remaining: 20.3min
[Parallel(n_jobs=12)]: Done   6 out of  15 | elapsed:  7.4min remaining: 11.1min
[Parallel(n_jobs=12)]: Done   8 out of  15 | elapsed:  7.5min remaining:  6.5min
[Parallel(n_jobs=12)]: Done  10 out of  15 | elapsed:  7.5min remaining:  3.7min
[Parallel(n_jobs=12)]: Done  12 out of  15 | elapsed:  7.5min remaining:  1.9min
[Parallel(n_jobs=12)]: Done  15 out of  15 | elapsed: 10.9min finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


{'mcfl__estimator__degree': 2, 'mcfl__estimator__kernel': 'poly'}
0.3669025064350609
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      5941
           1       0.00      0.00      0.00      1333
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00      3286
           4       0.00      0.00      0.00       644
           5       0.00      0.00      0.00       414
           6       0.00      0.00      0.00       239
           7       0.00      0.00      0.00       156
           8       0.00      0.00      0.00       267
           9       0.00      0.00      0.00       512
          10       0.00      0.00      0.00       878
          11       0.00      0.00      0.00       714
          12       0.00      0.00      0.00       123
          13       0.00      0.00      0.00       192
          14       0.00      0.00      0.00        88
          15       0.00      0.00      0.00       

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [86]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a28c4f6a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
     kernel='rbf', max_iter=-1, probability=False, random_state=None,
     shrinking=True, tol=0.001, verbose=False),
              n_jobs=None))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, en

In [106]:
from sklearn.svm import LinearSVC
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=LinearSVC()))
])

parameters = {
    'mcfl__estimator__C' : [1],
    'mcfl__estimator__loss' : ['hinge', 'squared_hinge'],
    'mcfl__estimator__max_iter' : [1000, 10000]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   35.1s
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:   35.2s remaining:  1.8min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:   35.5s remaining:   49.7s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:   36.5s remaining:   26.1s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:   36.8s remaining:   12.3s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   37.9s finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__C': [1], 'mcfl__estimator__loss': ['hinge', 'squared_hinge'], 'mcfl__estimator__max_iter': [1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [107]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

{'mcfl__estimator__C': 1, 'mcfl__estimator__loss': 'hinge', 'mcfl__estimator__max_iter': 1000}
0.6772156709563802
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5941
           1       0.78      0.58      0.67      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.69      0.72      3286
           4       0.65      0.20      0.31       644
           5       0.75      0.26      0.39       414
           6       0.79      0.13      0.22       239
           7       0.00      0.00      0.00       156
           8       0.67      0.25      0.37       267
           9       0.75      0.66      0.70       512
          10       0.79      0.76      0.77       878
          11       0.80      0.55      0.65       714
          12       0.76      0.45      0.56       123
          13       0.81      0.13      0.22       192
          14       0.57      0.18      0.28        88
          15       0.

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [96]:
cv.get_params()

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a28c4f6a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
              n_jobs=None))],
 'estimator__vect': CountVectorizer(analyzer='word', binary=False, deco

In [113]:
from sklearn.naive_bayes import MultinomialNB
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=MultinomialNB()))
])

parameters = {
    'mcfl__estimator__alpha' : [0.1, 1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of   9 | elapsed:   31.7s remaining:  1.9min
[Parallel(n_jobs=12)]: Done   3 out of   9 | elapsed:   31.9s remaining:  1.1min
[Parallel(n_jobs=12)]: Done   4 out of   9 | elapsed:   31.9s remaining:   39.9s
[Parallel(n_jobs=12)]: Done   5 out of   9 | elapsed:   31.9s remaining:   25.5s
[Parallel(n_jobs=12)]: Done   6 out of   9 | elapsed:   31.9s remaining:   16.0s
[Parallel(n_jobs=12)]: Done   7 out of   9 | elapsed:   31.9s remaining:    9.1s
[Parallel(n_jobs=12)]: Done   9 out of   9 | elapsed:   32.0s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   9 out of   9 | elapsed:   32.0s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__alpha': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [114]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

{'mcfl__estimator__alpha': 0.1}
0.5857248006170803
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      5941
           1       0.73      0.56      0.63      1333
           2       0.00      0.00      0.00        34
           3       0.69      0.68      0.68      3286
           4       0.59      0.16      0.25       644
           5       0.63      0.15      0.24       414
           6       1.00      0.01      0.02       239
           7       0.00      0.00      0.00       156
           8       0.65      0.18      0.28       267
           9       0.60      0.11      0.19       512
          10       0.69      0.32      0.44       878
          11       0.69      0.17      0.28       714
          12       0.58      0.11      0.19       123
          13       0.80      0.02      0.04       192
          14       1.00      0.01      0.02        88
          15       0.00      0.00      0.00       259
          16       0.72      0

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [111]:
cv.get_params()

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a28c4f6a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
              n_jobs=None))],
 'estimator__vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
       

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.